In [8]:
!pip install langchain

news = "51866453537741018957b8f780932298"
groq = "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"

In [9]:
from langchain.chat_models import ChatOpenAI # Import the ChatOpenAI class
import os
import json
import getpass
import requests
from datetime import datetime
from sentence_transformers import SentenceTransformer
from langchain_groq import ChatGroq
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter

# API Key Setup
NEWS_API_KEY = os.environ.get("NEWS_API_KEY") or getpass.getpass("Enter NewsAPI key: ")
GROQ_API_KEY = os.environ.get("GROQ_API_KEY") or getpass.getpass("Enter Groq API key: ")


Enter NewsAPI key: ··········
Enter Groq API key: ··········


In [11]:



from langchain_groq import ChatGroq


llm = ChatGroq(groq_api_key=GROQ_API_KEY, model_name="llama3-8b-8192", temperature=0.7)  # Use a valid model
# Embedding Model (Hugging Face Sentence Transformers)
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")



In [12]:

# User preferences file
PREF_FILE = "user_prefs.json"

# Load or initialize user preferences
def load_prefs():
    if os.path.exists(PREF_FILE):
        with open(PREF_FILE, 'r') as f:
            return json.load(f)
    return {"topics": [], "history": []}

def save_prefs(prefs):
    with open(PREF_FILE, 'w') as f:
        json.dump(prefs, f)

# Fetch news articles from NewsAPI
def fetch_news(topic):
    url = f"https://newsapi.org/v2/everything?q={topic}&apiKey={NEWS_API_KEY}&language=en"
    response = requests.get(url)
    if response.status_code == 200:
        articles = response.json()["articles"]
        return [{"title": a["title"], "content": a["content"] or a["description"]} for a in articles[:5]]
    return []

# Vectorize articles using SentenceTransformers

############33
def vectorize_articles(articles):
    texts = [f"{a['title']}\n{a['content']}" for a in articles]
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    chunks = text_splitter.create_documents(texts)

    # Generate text-embedding pairs
    text_embedding_pairs = [(chunk.page_content, embedding_model.encode(chunk.page_content)) for chunk in chunks]

    return FAISS.from_embeddings(text_embedding_pairs, embedding_model)



###################
# Summary prompts
brief_prompt = PromptTemplate(
    input_variables=["text"],
    template="Summarize this news article in 1-2 sentences:\n\n{text}"
)

detailed_prompt = PromptTemplate(
    input_variables=["text"],
    template="Provide a detailed paragraph summary of this news article:\n\n{text}"
)

# Summary chains
brief_chain = LLMChain(llm=llm, prompt=brief_prompt)
detailed_chain = LLMChain(llm=llm, prompt=detailed_prompt)

# Generate summary using Groq
def generate_summary(text, summary_type="brief"):
    chain = brief_chain if summary_type == "brief" else detailed_chain
    return chain.invoke({"text": text})  # ✅ Updated method


In [ ]:

# Main CLI
def main():
    prefs = load_prefs()

    while True:
        print("\nNews Summarization App")
        print("1. Search for news")
        print("2. Add topic preference")
        print("3. View saved topics")
        print("4. View search history")
        print("5. Exit")

        choice = input("Enter your choice (1-5): ")

        if choice == "1":
            topic = input("Enter news topic: ")
            articles = fetch_news(topic)
            if not articles:
                print("No articles found.")
                continue

            vector_store = vectorize_articles(articles)
            prefs["history"].append({"topic": topic, "date": str(datetime.now())})
            save_prefs(prefs)

            print("\nArticles found:")
            for i, article in enumerate(articles, 1):
                print(f"{i}. {article['title']}")

            article_num = int(input("Select article number to summarize: ")) - 1
            if 0 <= article_num < len(articles):
                text = f"{articles[article_num]['title']}\n{articles[article_num]['content']}"
                summary_type = input("Choose summary type (brief/detailed): ").lower()
                summary = generate_summary(text, summary_type)
                print(f"\nSummary:\n{summary}")

        elif choice == "2":
            topic = input("Enter topic to save: ")
            if topic not in prefs["topics"]:
                prefs["topics"].append(topic)
                save_prefs(prefs)
                print("Topic saved!")

        elif choice == "3":
            print("\nSaved topics:")
            for topic in prefs["topics"]:
                print(f"- {topic}")

        elif choice == "4":
            print("\nSearch history:")
            for entry in prefs["history"]:
                print(f"- {entry['topic']} ({entry['date']})")

        elif choice == "5":
            print("Goodbye!")
            break

        else:
            print("Invalid choice. Try again.")

if __name__ == "__main__":
    main()


News Summarization App
1. Search for news
2. Add topic preference
3. View saved topics
4. View search history
5. Exit



Articles found:
1. How to Watch Team USA vs. Canada in the NHL 4 Nations Face-Off Tonight
2. Amazon Boycott Starts Today; Includes Ring, Twitch, Whole Foods
3. Freedom of speech is ‘on the line’ in a pivotal Dakota Access Pipeline trial
4. Man City's Rodri returns to individual training
5. Underdog Finland aims to spoil Canada-USA II

Summary:
{'text': 'Here is a 2-sentence summary of the news article:\n\nThe 4 Nations Face-Off final between Team USA and Canada will take place on Thursday, February 20th at 8pm ET (5pm PT). The game will air on ESPN and can be watched live.'}

News Summarization App
1. Search for news
2. Add topic preference
3. View saved topics
4. View search history
5. Exit

Saved topics:
- football

News Summarization App
1. Search for news
2. Add topic preference
3. View saved topics
4. View search history
5. Exit

Search history:
- football (2025-03-15 16:35:35.166585)
- USA (2025-03-15 16:49:43.323282)

News Summarization App
1. Search for news
2. Add topic prefe


Articles found:
1. The Lunar Economy Is Coming
2. Infinity Nikki’s spooky new event will put my cheapskate skills to the test
3. Madame Web Is the Razzies’ Worst Picture of 2024? In a Joker 2 Economy?
4. Europe's Biggest Economy is in Trouble
5. How the British Broke Their Own Economy

Summary:
{'text': 'Here is a detailed paragraph summary of the news article:\n\nThe United Kingdom, the cradle of the Industrial Revolution, is now facing the opposite problem: severe energy shortages and deep affordability crises. According to a new report titled "Foundations", the country is struggling to recover from its own economic mismanagement. The report highlights the UK\'s failure to invest in its energy infrastructure, leading to frequent power outages and supply chain disruptions. Furthermore, the country\'s affordability crisis is exacerbated by high living costs, stagnating wages, and a lack of affordable housing, leaving many citizens struggling to make ends meet. The situation is particu